# Run this first cell to define all the functions:

In [2]:
# requirements are the folloing:
import math
import fractions
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np

def convertFrac(arr):
    return [fractions.Fraction(i) for i in arr]
def convertBack(arr):
    return [i.numerator/i.denominator for i in arr]

def elemProd(arr1, arr2, sumB=True):
    if not sumB:
        return [i*j for (i, j) in zip(arr1, arr2)]
    return sum([i*j for (i, j) in zip(arr1, arr2)])

def elemSq(arr):
    return [i*i for i in arr]

def convertListToDecimal(listA):
    return [frac if type(frac) == float else (frac if frac.denominator != 1 else frac.numerator) for frac in listA]

def checkForShortSelling(sigma1, sigma2, p12):
    if sigma1 >= sigma2:
        if p12 == 1:
            print("p12 = 1, there exist a deasable set with short selling s.t. sigmaV = 0.  each portfolio has the same sigmaV when sigma1 = sigma2")
        elif (sigma1/sigma2) < p12 < 1:
            print(f"feasable set with short selling, s.t. sigmaV < sigma1 {round(sigma1, 5)}, but without shortselling if sigmaV >= sigma1 {round(sigma1, 5)}")
        elif p12 == sigma1/sigma2:
            print("no short selling")
        elif -1 < p12 < sigma1/sigma2:
            print("feasable set without short selling")
        elif p12 == -1:
            print("there's a feasabale set without short selling, s.t. sigmaV < sigma1")
        else:
            print("wha?")
    else:
        checkForShortSelling(sigma2, sigma1, p12)

def makeCMatrix(correlationList, stdDevList):
    matrixSize = len(stdDevList)
    correlationArr = [[0 for _ in range(matrixSize)] for _ in range(matrixSize)]
    for i in range(matrixSize):
        for j in range(matrixSize):
            if i==j:
                correlationArr[i][j] = stdDevList[i]* stdDevList[i]
            else:    
                correlationArr[i][j] = findCorrelation(i+1, j+1, correlationList) * (stdDevList[i] * stdDevList[j])
    return correlationArr

def findCorrelation(i, j, correlation):
    for val in correlation:
        if val[0] == [i, j] or val[0] == [j, i]:
            return val[1]
                
def meanV(weights, means):
    return sum(w * m for (w, m) in zip(weights, means))

def find_wCwT(w, c):
    wC = np.matmul(w, c)
    print("intermediate calculation: w*c = ", wC)
    return np.matmul(wC, np.array(w).T)
    
def makeM(m, cInv, u):
    m, u = np.array(m), np.array(u)
    mCInv = np.matmul(m, cInv)
    uCInv = np.matmul(u, cInv)
    print("M*C^-1 = ", mCInv)
    print("u*C^-1 = ", uCInv)
    
    mCInv_mT = np.matmul(mCInv, m.T)
    mCInv_uT = np.matmul(mCInv, u.T)
    uCInv_mT = np.matmul(uCInv, m.T)
    uCInv_uT = np.matmul(uCInv, u.T)
    
    M = [[mCInv_mT, uCInv_mT],
        [mCInv_uT, uCInv_uT]]
    return np.array(M)

def minWeights(u, cInv):
    return np.matmul(u, cInv)/ (np.matmul(np.matmul(u, cInv), np.array(u).T))


# 2 securities (K1 and K2), 3 scenarios ==> find everthing else

In [2]:
p = ["0.2", "0.4", "0.4"]
k1 = ["-0.1", "0","0.2"]
k2 = ["0.05", "0.3", "-0.05"]

p = convertFrac(p)
k1 = convertFrac(k1)
k2 = convertFrac(k2)
E1 = elemProd(p, k1)
E2 = elemProd(p, k2)
E_k1sq = elemProd(p, elemSq(k1))
E_k2sq = elemProd(p, elemSq(k2))
var_k1 = E_k1sq - E1*E1
var_k2 =  E_k2sq - E2*E2
sigmak1 = fractions.Fraction(math.sqrt(var_k1))
sigmak2 = fractions.Fraction(math.sqrt(var_k2))

Ek1k2 = elemProd(elemProd(k1, k2, False), p)
c12 = Ek1k2 - E1*E2
p12 = c12 / (sigmak1*sigmak2)


w1 = (var_k2 - c12) / (var_k1 + var_k2 - 2*c12)
w2 = 1-w1
wfraction = w1/w2
wfraction2 = w2/w1



answers = [E1, E2, E_k1sq, E_k2sq, var_k1, var_k2, sigmak1, sigmak2, Ek1k2, c12, p12, w1, w2, wfraction, wfraction2]
equations = ["E[K1]", "E[K2]", "E[K1^2]", "E[K2^2]", "E(K1^2)- E[K1]^2","E(K2^2)- E[K2]^2", "sqrt(Var(K1))", "sqrt(Var(K2))", 
             "E[K1*K2]", "Cov(K1, K2)", "p12", "w1", "w2", "stdDev(K1)/stdDev(K2)", "stdDev(K2)/stdDev(K1)"]
stringVal = ["E[K1]", "E[K2]", "E[K1^2]", "E[K2^2]", "Var(K1)","Var(K2)", "StdDev(K1)", "StdDev(K2)", 
             "E[K1*K2]", "Cov(K1, K2)", "p12", "w1", "w2", "stdDev(K1)/stdDev(K2)", "stdDev(K2)/stdDev(K1) \n **if std(K1)>std(K2)"]

decimalAnswers = convertBack(answers)

dataLists = [stringVal, equations, decimalAnswers, answers]
df = pd.DataFrame(dataLists, index=["Variable", "Formula", "Value", "fractional Value"]).T
print("")
checkForShortSelling(sigmak1, sigmak2, p12)
df


feasable set without short selling


,Variable,Formula,Value,fractional Value
0,E[K1],E[K1],0.06,3/50
1,E[K2],E[K2],0.11,11/100
2,E[K1^2],E[K1^2],0.018,9/500
3,E[K2^2],E[K2^2],0.0375,3/80
4,Var(K1),E(K1^2)- E[K1]^2,0.0144,9/625
5,Var(K2),E(K2^2)- E[K2]^2,0.0254,127/5000
6,StdDev(K1),sqrt(Var(K1)),0.12,1080863910568919/9007199254740992
7,StdDev(K2),sqrt(Var(K2)),0.159374,717755671473763/4503599627370496
8,E[K1*K2],E[K1*K2],-0.005,-1/200
9,"Cov(K1, K2)","Cov(K1, K2)",-0.0116,-29/2500


# HW8 question 1

In [3]:
# w1, w2, ...., wm
weights = [0.3, 0.1, 0.6]
means = [0.08, 0.1, 0.06]
stdDevs = [0.15, 0.05, 0.12]

# each entry represent correlation of (i, j) ans stored in the form [(i, j), value]
correlation = [
    [[1, 2], 0.3],
    [[2, 3], 0],
    [[1, 3], -0.2],
]


mu_V = meanV(weights, means) 
cMatrix = makeCMatrix(correlation, stdDevs)
wCwT = find_wCwT(weights, cMatrix)
np.set_printoptions(suppress=True)
print("Expected value of portfolio is:", mu_V)
print("Correlation matrix is:")
print(np.array(cMatrix))
print("w*C*wT is: ", wCwT)
print("stdDev of portfolio is", math.sqrt(wCwT))
print("\n", "*"* 20, "\n")
cInverse = np.linalg.inv(cMatrix)
print("here is the inverse of C:")
print(cInverse)
m = np.array(means)
u = np.array([1 for _ in means])
print("m is:", m)
print("u is:", u)
print("calculating M")
Mmatrix = makeM(m, cInverse, u)
print("here is M:")
print(Mmatrix)
print("here is M inverse:")
MmatrixInv = np.linalg.inv(Mmatrix)
print(MmatrixInv)

intermediate calculation: w*c =  [0.004815 0.000925 0.00756 ]
Expected value of portfolio is: 0.07
Correlation matrix is:
[[ 0.0225   0.00225 -0.0036 ]
 [ 0.00225  0.0025   0.     ]
 [-0.0036   0.       0.0144 ]]
w*C*wT is:  0.006072999999999999
stdDev of portfolio is 0.07792945527847606

 ******************** 

here is the inverse of C:
[[ 51.08556833 -45.97701149  12.77139208]
 [-45.97701149 441.37931034 -11.49425287]
 [ 12.77139208 -11.49425287  72.63729246]]
m is: [0.08 0.1  0.06]
u is: [1 1 1]
calculating M
M*C^-1 =  [ 0.25542784 39.77011494  4.23052363]
u*C^-1 =  [ 17.87994891 383.90804598  73.91443167]
here is M:
[[  4.25127714  44.25606641]
 [ 44.25606641 475.70242656]]
here is M inverse:
[[ 7.46276714 -0.69428428]
 [-0.69428428  0.06669357]]


# HW 7 Part 2:

In [5]:
m = [0.2, 0.13, 0.17]
stdDevs = [0.25, 0.28, 0.20]
correlation= [
    [[1,2], 0.25],
    [[2,3], 0],
    [[3,1], 0.15]]

m = np.array(m)
u = np.array([1 for _ in means])

print("m is:", m)
print("u is:", u)
cMatrix = makeCMatrix(correlation, stdDevs)
print("\nCorrelation matrix is:")
print(np.array(cMatrix))
cInverse = np.linalg.inv(cMatrix)
print("\nhere is the inverse of C:")
print(cInverse)
print("\ncalculating M:")
Mmatrix = makeM(m, cInverse, u)
print("\nhere is M:")
print(Mmatrix)
print("\nhere is M inverse:")
MmatrixInv = np.linalg.inv(Mmatrix)
print(MmatrixInv)
minimumWeights = minWeights(u, cInverse)
print("\nThe weight in minimum variance portfolio is:")
print(minimumWeights)
print("\nThe Expected return is:")
Emin = elemProd(minimumWeights, m)
print(Emin)
print("\Variance is:")
wCwT = find_wCwT(minimumWeights, cMatrix)
print("wCwT = Varinace = ")  
print(wCwT)
stdDevMin = math.sqrt(wCwT)
print("\nThe Std.Dev of the portfolio is:")
print(stdDevMin)

m is: [0.2  0.13 0.17]
u is: [1 1 1]

Correlation matrix is:
[[0.0625 0.0175 0.0075]
 [0.0175 0.0784 0.    ]
 [0.0075 0.     0.04  ]]

here is the inverse of C:
[[17.4863388  -3.90320062 -3.27868852]
 [-3.90320062 13.62635218  0.73185012]
 [-3.27868852  0.73185012 25.6147541 ]]

calculating M:
M*C^-1 =  [2.43247463 1.11520018 3.79391101]
u*C^-1 =  [10.30444965 10.45500167 23.06791569]

here is M:
[[ 1.27643582  7.34158581]
 [ 7.34158581 43.82736701]]

here is M inverse:
[[21.44260148 -3.59188127]
 [-3.59188127  0.62449804]]

The weight in minimum variance portfolio is:
[0.2351145  0.23854962 0.52633588]

The Expected return is:
0.16751145038167942
\Variance is:
intermediate calculation: w*c =  [0.02281679 0.02281679 0.02281679]
wCwT = Varinace = 
0.022816793893129773

The Std.Dev of the portfolio is:
0.1510522886060644


# HW 8, Problem 3:

In [4]:
m = [0.2, 0.1, 0.2]
stdDevs = [0.25, 0.2, 0.25]
correlation= [
    [[1,2], 0.25],
    [[2,3], 0.05],
    [[3,1], 0.15]]

m = np.array(m)
u = np.array([1 for _ in m])

print("m is:", m)
print("u is:", u)
cMatrix = makeCMatrix(correlation, stdDevs)
print("\nCorrelation matrix is:")
print(np.array(cMatrix))
cInverse = np.linalg.inv(cMatrix)
print("\nhere is the inverse of C:")
print(cInverse)
print("\ncalculating M:")
Mmatrix = makeM(m, cInverse, u)
print("\nhere is M:")
print(Mmatrix)
print("\nhere is M inverse:")
MmatrixInv = np.linalg.inv(Mmatrix)
print(MmatrixInv)
minimumWeights = minWeights(u, cInverse)
print("\nThe weight in minimum variance portfolio is:")
print(minimumWeights)
print("\nThe Expected return is:")
Emin = elemProd(minimumWeights, m)
print(Emin)
print("\Variance is:")
wCwT = find_wCwT(minimumWeights, cMatrix)
print("wCwT = Varinace = ")  
print(wCwT)
stdDevMin = math.sqrt(wCwT)
print("\nThe Std.Dev of the portfolio is:")
print(stdDevMin)

m is: [0.2 0.1 0.2]
u is: [1 1 1]

Correlation matrix is:
[[0.0625   0.0125   0.009375]
 [0.0125   0.04     0.0025  ]
 [0.009375 0.0025   0.0625  ]]

here is the inverse of C:
[[17.41882674 -5.29331514 -2.40109141]
 [-5.29331514 26.67121419 -0.2728513 ]
 [-2.40109141 -0.2728513  16.37107776]]

calculating M:
M*C^-1 =  [2.47421555 1.55388813 2.76671214]
u*C^-1 =  [ 9.72442019 21.10504775 13.69713506]

here is M:
[[ 1.20357435  6.79481583]
 [ 6.79481583 44.526603  ]]

here is M inverse:
[[ 5.99963235 -0.91555147]
 [-0.91555147  0.16217279]]

The weight in minimum variance portfolio is:
[0.21839574 0.47398738 0.30761689]

The Expected return is:
0.15260126233225074
\Variance is:
intermediate calculation: w*c =  [0.02245848 0.02245848 0.02245848]
wCwT = Varinace = 
0.022458483975733808

The Std.Dev of the portfolio is:
0.1498615493571777
